In [0]:
#Datasets used here:
# mosaicml/dolly_hhrlhf
# b-mc2/sql-create-context (only a ~4.2k subset)
# code alpaca 20k -cc license
# qwedsacf/grade-school-math-instructions - not using because it's in HELM, decided to use it because no point benchmarking

In [0]:
from datasets import load_dataset , Dataset, concatenate_datasets 
import numpy as np
import pandas as pd
import random

In [0]:
capaca = load_dataset("HuggingFaceH4/CodeAlpaca_20K")
capaca_train_df = pd.DataFrame(capaca['train'])
capaca_test_df = pd.DataFrame(capaca['test'])
capaca_df = pd.concat([capaca_train_df, capaca_test_df])
display(capaca_df)

In [0]:
template = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{}

### Response:\n"""
capaca_df['prompt'] = capaca_df["prompt"].apply(lambda x: template.format(x))

capaca_df.rename(columns={'completion': 'response'}, inplace=True)
capaca_df = capaca_df[['prompt', 'response']]
display(capaca_df)

In [0]:
drhlf = load_dataset("mosaicml/dolly_hhrlhf")
drlhf_train_df = pd.DataFrame(drhlf['train'])
drlhf_test_df = pd.DataFrame(drhlf['test'])
drlhf_df = pd.concat([drlhf_train_df, drlhf_test_df])
drlhf_df.drop_duplicates(inplace=True)
display(drlhf_df), drlhf_df.shape


In [0]:
text_to_sql = load_dataset("b-mc2/sql-create-context")
txsql_df = pd.DataFrame(text_to_sql)
display(txsql_df)

In [0]:
import json 
dsdf_ = pd.json_normalize(txsql_df['train'])
#Dropping all examples where no definite table name is given i.e. most of WikiSQL
dsdf_ = dsdf_[~dsdf_['answer'].str.contains('FROM table_')]
dsdf_.drop_duplicates(inplace=True)
display(dsdf_), dsdf_.shape

In [0]:
dsdf_['question'] = 'Generate SQL query: '+ dsdf_['question']
dsdf_['context'] = dsdf_['context'].apply(lambda x: x.replace("CREATE TABLE", "").replace("VARCHAR", "STRING").strip())
dsdf_['Instruction'] = dsdf_['question']+ ', given the following schema: ' + dsdf_['context']
dsdf_.drop(columns=['question', 'context'], inplace=True)
display(dsdf_)

In [0]:
template = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{}

### Response:\n"""

In [0]:
dsdf_['prompt'] = dsdf_["Instruction"].apply(lambda x: template.format(x))

In [0]:
display(dsdf_), dsdf_.shape

In [0]:
dsdf_.drop(columns = ['Instruction'], inplace=True)
dsdf_.rename(columns={'answer': 'response'}, inplace=True)
dsdf_ = dsdf_[['prompt', 'response']]
display(dsdf_)

In [0]:
#qwedsacf/grade-school-math-instructions
mathinstr = load_dataset("qwedsacf/grade-school-math-instructions")
mathinstr_df = pd.DataFrame(mathinstr['train'])
template = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{}

### Response:\n"""
mathinstr_df['prompt'] = mathinstr_df["INSTRUCTION"].apply(lambda x: template.format(x))

mathinstr_df.drop(columns = ['INSTRUCTION','SOURCE'], inplace=True)
mathinstr_df.rename(columns={'RESPONSE': 'response'}, inplace=True)
mathinstr_df = mathinstr_df[['prompt', 'response']]
display(mathinstr_df)

In [0]:
dolly_llama_df = pd.concat([drlhf_df, mathinstr_df, capaca_df, dsdf_ ], ignore_index=True).sample(frac=1, random_state=42)
dolly_llama_df.drop_duplicates(inplace=True)
dolly_llama_df.shape

In [0]:
dolly_llama_df['response'] = dolly_llama_df['response'] +  "\n### End"

In [0]:
dolly_llama_df.drop_duplicates(inplace=True)
display(dolly_llama_df), dolly_llama_df.shape

In [0]:
%sql
CREATE DATABASE IF NOT EXISTS dolly_llama;
USE dolly_llama;

In [0]:
%sql
DROP TABLE dolly_llama_instruct;

In [0]:
spark.createDataFrame(dolly_llama_df).write.saveAsTable('dolly_llama_instruct')

In [0]:
%sql
SELECT * FROM dolly_llama_instruct